# TearSheet.ipynb

- doesn't actually generate a tearsheet like a classic print [S&P or Value Line tearsheet](https://financetrain.com/what-is-a-tear-sheet) on a company
- asks Perplexity a few basic questions about the company
- gets info from AlphaVantage
- opens a bunch of browser tabs from major services about the company
- could potentially save those and scrape them to make a tearsheet.


Updated version
- profile - get info from perplexity , other company profiles, openbb, sec filings, wikipedia generate a description of the company, focus on business model history of the company
- news
  - get latest news
  - scrape news using scraper - grab headlines matching pattern
  - filter stuff by how relevant to the company
- chart
- key ratios, top holders
- generate tear sheet using tools
- question answering chatbot using tools


In [1]:
import dotenv
import sys
import os
import re
from datetime import datetime, timedelta
import time

import pandas as pd

import requests
import json

import openai
from openai import OpenAI

import IPython
from IPython.display import HTML, Image, Markdown, display

from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service


In [2]:
company = "Tesla"
symbol = "TSLA"


In [3]:
dotenv.load_dotenv()



True

In [4]:
system_prompt = """
You will act as a securities analyst and investment advisor with deep knowledge of financial markets, 
securities analysis, portfolio management. You will maintain a professional yet engaging tone, 
in the style of a senior investment bank research analyst."
"""

user_prompt = f"""You will focus on {company} ({symbol}), and provide a comprehensive analysis covering the following aspects:

Company Profile: An overview of {company}, including its lines of business, history, and recent key developments.

Major News: Significant events related to {company} or its industry impacting its stock.

Financial Performance: Recent earnings reports and stock performance compared to expectations, changes in dividends or stock buybacks.

Analyst Coverage: summarize recent changes to analysts' ratings noting which analyst and firms made upgrades or downgrades; summarize any recent short seller reports noting the firm and analyst.

Product Announcements: Launch of new products, strategic initiatives, or restructurings.

Strategic Moves: Information on deals, partnerships, mergers, acquisitions, divestitures, joint ventures, and major new business and revenue.

Securities Offerings: Announcements related to stock or bond issuances, buybacks, special dividends, or stock splits.

Management Changes: Significant personnel changes within {company}.

Stock Price Movements: Notable stock price changes and their reasons.

Timeline and Future Outlook: A timeline of these events """


In [5]:
perplexity_url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "pplx-7b-online",
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

perplexity_headers = {
    "Authorization": f"Bearer {os.getenv('PERPLEXITY_API_KEY')}",
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_str = response.json()['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
display(Markdown(response_str))


## Company Profile

Tesla, Inc. is a leading company in the design, development, manufacture, and sale of electric vehicles (EVs) and energy generation and storage systems. The company operates under two primary segments: Automotive and Energy Generation and Storage. The Automotive segment includes the creation, production, marketing, and leasing of electric vehicles, as well as the sale of automotive regulatory credits. The Energy Generation and Storage segment focuses on the design, manufacture, installation, sale, and lease of stationary energy storage products and solar energy systems. Additionally, the segment sells solar energy systems incentives. Tesla's headquarters is located in Austin, TX.

## Major News

Recently, Tesla's stock experienced a 15% surge this week following the release of its first-quarter earnings. A former high-ranking executive, Drew Baglino, sold over 1 million shares of TSLA for approximately \\$181.5 million. Baglino, who previously served as senior vice president of powertrain and energy at Tesla, announced his departure from the company on April 15. Elon Musk, Tesla's CEO, also announced plans to reduce the workforce by more than 10%, aiming to prepare for the next phase of growth.

## Financial Performance

Tesla reported its first-quarter earnings and revenue on Tuesday. The stock price advanced around 0.7% during Friday's premarket trading and has gained over 15% on the week. Despite the weekly gain, Tesla shares remain approximately 3.3% below their April value. Last week, Tesla stock plummeted 14% and dipped below April 2023 lows.

## Analyst Coverage

Currently, the analyst ratings for Tesla, Inc. (TSLA) are:

- Buy: [percentage]%
- Hold: [percentage]%
- Sell: [percentage]%

(Note: Percentages may vary)

## Product Announcements

No significant product announcements have been made recently.

## Strategic Moves

Tesla's stock price is currently testing resistance at its 10-week moving average. The company has faced challenges in reclaiming this level throughout much of 2024. A decisive break above resistance could signal a positive development. Conversely, if Tesla stock fails to regain its 10-week moving average, this declining marker may continue to function as a barrier to the stock price.

## Securities Offerings

No significant securities offerings have been announced recently.

## Management Changes

Drew Baglino, a former high-ranking executive at Tesla, sold over 1 million shares of TSLA for approximately \\$181.5 million. Baglino, who previously served as senior vice president of powertrain and energy at Tesla, announced his departure from the company on April 15. Elon Musk, Tesla's CEO, also announced plans to reduce the workforce by more than 10%, aiming to prepare for the next phase of growth.

## Stock Price Movements

Tesla stock experienced a 15% surge this week following the release of its first-quarter earnings. The stock price advanced around 0.7% during Friday's premarket trading and has gained over 15% on the week. Despite the weekly gain, Tesla shares remain approximately 3.3% below their April value. Last week, Tesla stock plummeted 14% and dipped below April 2023 lows.

## Timeline and Future Outlook

Tesla's stock price has been volatile in recent weeks, with a 15% surge this week following the release of its first-quarter earnings. The company has faced challenges in reclaiming its 10-week moving average, which may continue to function as a barrier to the stock price. The future outlook for Tesla stock depends on its ability to overcome these challenges and continue its growth trajectory.

In [ ]:
stuff to focus on
- is it growing
- is it quality, does it have a moat and consistent earnings
- are customers loyal , 
- are employees and management long-tenured
- is capital intensive
- does it have a lot of debt
- does it have quality management that are operators as opposed to promoters or financiers

- 

In [6]:
user_prompt = f"""What are the current analyst ratings on {company} ({symbol})? 
Which short sellers issued reports on {company}, if any? Summarize any notable analyst reports."""

payload = {
    "model": "pplx-7b-online",
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_str = response.json()['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
display(Markdown(response_str))


As of April 26, 2024, the current analyst ratings on Tesla (TSLA) are not explicitly mentioned in the search results provided. However, based on the information available, we can infer that the short interest ratio for Tesla is 1.75 days to cover, indicating strong positive sentiment about the stock and a lack of short sellers. This suggests that the analyst ratings for Tesla are generally positive, as short interest ratios below 4 are typically associated with bullish sentiment.

There is no mention of any short sellers issuing reports on Tesla in the search results.

Notable analyst reports on Tesla include those from MarketBeat and Barchart. MarketBeat offers a suite of research tools that includes news, buy/sell ratings, SEC filings, insider transactions, and personalized stock ideas based on a user's portfolio. Barchart provides stock analyst ratings on a scale of 1 to 5, with 1 being a strong buy and 5 being a strong sell. However, the specific ratings for Tesla are not mentioned in the search results.

In [7]:
user_prompt = f"""Have there been any notable news stories about {company} ({symbol}) in the past year, including management profiles and investigative reports? 
For any notable stories, provide the date of publication and the media that reported them."""

payload = {
    "model": "pplx-7b-online",
    "messages": [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
}

response = requests.post(perplexity_url, json=payload, headers=perplexity_headers)
response_str = response.json()['choices'][0]['message']['content']
response_str = response_str.replace("$", "\\\$")
display(Markdown(response_str))


In the past year, several notable news stories have emerged about Tesla (TSLA), including investigations and management profiles. Here are some of the key events and reports:

1. **Justice Department Investigation**: In October 2023, the Justice Department expanded its investigation into Tesla beyond the electric vehicle maker's partially automated driving systems. The investigation now includes "personal benefits, related parties, vehicle range, and personnel decisions." Tesla disclosed this in a quarterly report filed with the Securities and Exchange Commission, suggesting that prosecutors may be investigating Tesla CEO Elon Musk and the company's representations about its vehicles.

2. **Recall Investigation**: In July 2023, Tesla faced an investigation over a recall for 2 million cars due to issues with its Autopilot system.

3. **Secret Team to Suppress Driving Range Complaints**: In July 2023, Tesla was found to have created a secret team to suppress thousands of driving range complaints. The team was tasked with diverting customers who did not require in-person service and offering tips on extending range by changing driving habits.

4. **NHTSA Investigation**: In August 2023, the National Highway Traffic Safety Administration (NHTSA) opened a probe into Tesla's Autopilot system in response to a dozen complaints of reported loss of steering control.

5. **Executive Probes**: In August 2023, Tesla faced multiple US investigations, with current and former executives subpoenaed.

These investigations and reports highlight the ongoing scrutiny of Tesla's operations and management, particularly in the areas of vehicle safety and consumer protection.

In [8]:
# AlphaVantage overview
url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={os.environ['ALPHAVANTAGE_API_KEY']}"
r = requests.get(url)
data = r.json()
pd.DataFrame(data, index=[1]).transpose()


,1
Symbol,TSLA
AssetType,Common Stock
Name,Tesla Inc
Description,"Tesla, Inc. is an American electric vehicle an..."
CIK,1318605
Exchange,NASDAQ
Currency,USD
Country,USA
Sector,MANUFACTURING
Industry,MOTOR VEHICLES & PASSENGER CAR BODIES


In [9]:
exchange = data['Exchange']

morningstarmap = {'NYSE': 'xnys',
                  'NASDAQ': 'xnas'
                 }


In [10]:
now = datetime.now()
end_date = now.strftime('%Y%m%dT%H%M')
sdate = now - timedelta(days=2)
# sdate = datetime(now.year, now.month-1, now.day)
start_date = sdate.strftime('%Y%m%dT%H%M')
start_date, end_date

('20240424T1046', '20240426T1046')

In [11]:
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={os.environ["ALPHAVANTAGE_API_KEY"]}&time_from={start_date}&time_to={end_date}'

r = requests.get(url)
data = r.json()

for item in data['feed']:
    markdown_str = ""
    date_object = datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S")
    display_title = item['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item['summary'].replace("$", "\\\$")
    markdown_str += f"[{str(date_object)} {display_title}]({item['url']})\n {description}"
    display(Markdown(markdown_str))
    

[2024-04-26 14:25:22 Market Clubhouse Morning Memo - April 26th, 2024  ( Trade Strategy For SPY, QQQ, AAPL, MSFT, NVDA, GOOGL, META, And TSLA )  - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) ](https://www.benzinga.com/trading-ideas/24/04/38461474/market-clubhouse-morning-memo-april-26th-2024-trade-strategy-for-spy-qqq-aapl-msft-nvda-googl-meta)
 Good Morning Traders! In today's Market Clubhouse Morning Memo, we will discuss SPY, QQQ, AAPL, MSFT, NVDA, GOOGL, META, and TSLA. Our proprietary formula, exclusive to Market Clubhouse, dictates these price levels. This dynamic equation takes into account price, volume, and options flow.

[2024-04-26 14:22:51 'Meta Actually Has Major Profit Margins Unlike Tesla': Redditor Reacts To Stock Drop Following Q1 Earnings - Meta Platforms  ( NASDAQ:META ) ](https://www.benzinga.com/news/earnings/24/04/38461410/meta-actually-has-major-profit-margins-unlike-tesla-redditor-reacts-to-stock-drop-following-q1-earn)
 Meta Platforms Inc META, formerly Facebook, recently reported a revenue and earnings beat for Q1 2024 but issued disappointing guidance for the future. This news caused the stock to drop 11% in extended trading, overshadowing the positive quarterly results.

[2024-04-26 14:22:11 Stock Market Dodges A Bullet - Momo Gurus Proven Wrong - Microsoft  ( NASDAQ:MSFT ) ](https://www.benzinga.com/markets/24/04/38461403/stock-market-dodges-a-bullet-momo-gurus-proven-wrong)
 To gain an edge, this is what you need to know today. Please click here for an enlarged chart of SPDR S&P 500 ETF Trust SPY which represents the benchmark stock market index S&P 500 ( SPX ) . Headline PCE came at 0.3% vs. 0.3% consensus. Core PCE came at 0.3% vs. 0.3% consensus.

[2024-04-26 14:06:12 Nvidia, Meta Among Tech Stocks That Stand To Benefit From 'Lots Of Infrastructure Spending On AI,' Says Tesla Bull Ross Gerber - Alphabet  ( NASDAQ:GOOG ) , ASML Holding  ( NASDAQ:ASML ) ](https://www.benzinga.com/analyst-ratings/analyst-color/24/04/38460970/nvidia-meta-among-tech-stocks-that-stand-to-benefit-from-lots-of-infrastructure-spe)
 Tesla bull Ross Gerber has suggested that the tech sector's earnings reports are indicative of substantial AI infrastructure spending. What Happened: Gerber took to X, formerly Twitter, on Thursday to express his views on the tech sector's earnings reports.

[2024-04-26 12:49:00 Tesla Stock Tests Key Resistance Level As Former Top Executive Divests 1.14 Million Shares](https://www.investors.com/news/tesla-stock-tests-resistance-former-top-executive-divests-1-million-tsla-shares/)
 Tesla ( TSLA ) is looking to make a move above a key resistance level after gaining more than 15% this week following first-quarter earnings. A federal regulatory filing Thursday shows that a former top executive has unloaded more than 1 million shares.

...

[2024-04-25 18:46:39 Billionaire Ron Baron Calls The Bottom In Tesla Stock, Says Shares Will 'Go Up Huge' From Here: 'It's Like A Rubber Band' - Tesla  ( NASDAQ:TSLA ) ](https://www.benzinga.com/trading-ideas/long-ideas/24/04/38441703/billionaire-ron-baron-calls-the-bottom-in-tesla-stock-says-shares-will-go-up-huge-from-h)
 Billionaire investor Ron Baron believes Tesla Inc TSLA stock has fallen far enough and is now getting ready to rip higher from current levels. Why TSLA Is Down: Thursday on CNBC's "Squawk Box," Baron, a longtime Tesla bull, explained both why Tesla stock is down so much for the year and why he ...

[2024-04-25 18:45:14 Elon Musk once again richer than Mark Zuckerberg as fortunes reverse](https://www.business-standard.com/world-news/elon-musk-once-again-richer-than-mark-zuckerberg-as-fortunes-reverse-124042600010_1.html)
 Mark Zuckerberg's fortune plunged \\$20 billion as shares of Meta Platforms Inc. tumbled, allowing Tesla Inc.'s Elon Musk to cement his status as the world's third-richest billionaire. Meta shares fell as much as 16% Thursday in New York after the social media company said second-quarter sales were ...

[2024-04-25 16:09:00 Magnificent Seven Stocks To Buy And Watch](https://www.investors.com/research/magnificent-seven-stocks-to-buy-and-and-watch/)
 Magnificent Seven Stocks: Nvidia Stock Dives Below Key Level Investor's Business Daily ...

[2024-04-25 16:01:01 Alphabet set to report first-quarter results after market close](https://www.cnbc.com/2024/04/25/alphabet-set-to-report-first-quarter-results-after-market-close.html)
 Analysts expect a second straight quarter of revenue growth in the low teens as Alphabet rebounds from a historically weak period for expansion.

[2024-04-25 16:00:01 Microsoft set to report earnings after the bell](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html)
 Microsoft added artificial intelligence talent through a deal with startup Inflection in the quarter, as it worked to sell a productivity chatbot to clients.

In [12]:
from newsapi import NewsApiClient
NEWSAPI_API_KEY = os.environ['NEWSAPI_API_KEY']
api = NewsApiClient(api_key=NEWSAPI_API_KEY)
sources = api.get_sources()
pd.DataFrame(sources['sources'])

,id,name,description,url,category,language,country
0,abc-news,ABC News,"Your trusted source for breaking news, analysi...",https://abcnews.go.com,general,en,us
1,abc-news-au,ABC News (AU),"Australia's most trusted source of local, nati...",https://www.abc.net.au/news,general,en,au
2,aftenposten,Aftenposten,Norges ledende nettavis med alltid oppdaterte ...,https://www.aftenposten.no,general,no,no
3,al-jazeera-english,Al Jazeera English,"News, analysis from the Middle East and worldw...",https://www.aljazeera.com,general,en,us
4,ansa,ANSA.it,"Agenzia ANSA: ultime notizie, foto, video e ap...",https://www.ansa.it,general,it,it
...,...,...,...,...,...,...,...
123,wired,Wired,"Wired is a monthly American magazine, publishe...",https://www.wired.com,technology,en,us
124,wired-de,Wired.de,Wired reports on how emerging technologies aff...,https://www.wired.de,technology,de,de
125,wirtschafts-woche,Wirtschafts Woche,Das Online-Portal des führenden Wirtschaftsmag...,http://www.wiwo.de,business,de,de
126,xinhua-net,Xinhua Net,"中国主要重点新闻网站,依托新华社遍布全球的采编网络,记者遍布世界100多个国家和地区,地方频...",http://xinhuanet.com/,general,zh,zh


In [13]:
# Calculate the date 24 hours ago
date_from = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S')

# The base URL for the NewsAPI
url = 'https://newsapi.org/v2/everything'

# Parameters for the API request
parameters = {
    'q': symbol,  # Query term
    'from': date_from,  # Get news from the last 24 hours
    'sortBy': 'publishedAt',  # Sort by publication date
    'apiKey': NEWSAPI_API_KEY  # Your API key
}

# Make the request to the NewsAPI
response = requests.get(url, params=parameters)

# Check if the request was successful
if response.status_code == 200:
    # Get the response data
    data = response.json()
    
    # Print the articles
    markdown_str = ""
    for article in data['articles']:
        date_object = datetime.strptime(item['time_published'], "%Y%m%dT%H%M%S")
        display_title = article['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
        markdown_str += f"[{article['publishedAt']} {display_title}]({item['url']}) \n\n"

    display(Markdown(markdown_str))
else:
    print('Failed to retrieve data')

[2024-04-25T14:36:05Z Market Clubhouse Morning Memo - April 25th, 2024 (Trade Strategy For SPY, QQQ, AAPL, MSFT, NVDA, GOOGL, META, And TSLA)](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T14:00:00Z Rising 21% This Year, What Lies Ahead For Exxon Stock Following Q1 Earnings?](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T13:00:00Z Should You Pick General Electric Stock At \\$165?](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T12:48:27Z Dow Jones Futures Dive 400 Points On Weak GDP Data; Meta Plunges On Earnings](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T12:40:45Z Tesla (NASDAQ:TSLA) Price Target Cut to \\$180.00 by Analysts at Mizuho](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T12:40:44Z Tesla (NASDAQ:TSLA) Price Target Lowered to \\$162.00 at Truist Financial](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T12:40:44Z Tesla (NASDAQ:TSLA) Rating Reiterated by Needham & Company LLC](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T12:00:00Z What’s Next For JetBlue Stock After A Sharp 19% Fall Post Q1 Results?](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T12:00:00Z Mag 7 ETF Investors Eye Big Tech Earnings, PCE](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T11:56:08Z Is Tesla a Software Company?](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T11:16:44Z Tesla (NASDAQ:TSLA) Trading Up 1.8%](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T11:16:05Z Why Tesla’s stock price is surging](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 

[2024-04-25T11:00:00Z Is Kimberly-Clark Stock Fairly Valued At \\$135 After A Solid Q1?](https://www.cnbc.com/2024/04/25/microsoft-msft-q3-earnings-2024.html) 



In [14]:
now = datetime.now()
end_date = now.strftime('%Y-%m-%d')
# start_date = datetime.now() - timedelta(days=30)
sdate = datetime(now.year, now.month-1, now.day)
start_date = sdate.strftime('%Y-%m-%d')
start_date, end_date

('2024-03-26', '2024-04-26')

In [15]:
API_KEY = os.environ['NEWSFILTER_API_KEY']
API_ENDPOINT = "https://api.newsfilter.io/search?token={}".format(API_KEY)

# Define the news search parameters
queryString = f"symbols:{symbol} AND publishedAt:[{start_date} TO {end_date}]"

payload = {
    "queryString": queryString,
    "from": 0,
    "size": 10
}

# Send the search query to the Search API
response = requests.post(API_ENDPOINT, json=payload)

# Read the response
articles = response.json()


# Read the response
articles = response.json()

for item in articles['articles']:
    markdown_str = ""
    date_array = item['publishedAt'].split("T")
    datestr = date_array[0] + "T" + date_array[1][:8]
#     print(item['publishedAt'])
#     print(date_array)
#     print(datestr)
    date_object = datetime.strptime(datestr, "%Y-%m-%dT%H:%M:%S")
    display_title = item['title'].replace("$", "\\\$")  # so Markdown doesn't interpret as latex escape
    description = item['description'].replace("$", "\\\$")

    markdown_str += f"[{str(date_object)} {display_title}]({item['url']})\n {description}"
    display(Markdown(markdown_str))

[2024-04-26 13:42:00 Tesla’s AutoPilot Recall Gets a Government Probe. No, It’s Not a Stock Issue.](https://newsfilter.io/articles/teslas-autopilot-recall-gets-a-government-probe-no-its-not-a-stock-issue-ba82494103e20526a2e121375dc61de9)
 NHTSA is investigating Tesla’s AutoPilot system again. Investors don’t need to fret. 

[2024-04-26 10:40:00 Microsoft, Alphabet Earnings Show Markets Need AI After Fed Cut Delay. Here’s Why.](https://newsfilter.io/articles/microsoft-alphabet-earnings-show-markets-need-ai-after-fed-cut-delay-heres-why-17fe662515161685606e479f85f20a7c)
 

[2024-04-26 08:54:00 Tesla, GM, and Ford Had Quite a Week. All the Numbers That Matter.](https://newsfilter.io/articles/tesla-gm-and-ford-had-quite-a-week-all-the-numbers-that-matter-d86dc85ea033ac933783efdc14b22dec)
 Tesla ended the quarter with less than 50% BEV market share in the U.S. for the first time.

[2024-04-26 02:23:08 Former Tesla SVP Drew Baglino is selling \\$181.5 million worth of stock, SEC filing says](https://newsfilter.io/articles/former-tesla-svp-drew-baglino-is-selling-1815-million-worth-of-stock-sec-filing-says-0935229bc04ab116721dcd714a297961)
 Drew Baglino, who announced he'd be leaving Tesla on April 15, is selling 1.14 million shares of the electric car company, according to a filing on April 25.

[2024-04-25 17:10:14 144 Form - Report of proposed sale of securities - Tesla, Inc. (0001318605) (Subject)](https://newsfilter.io/articles/144-form---report-of-proposed-sale-of-securities---tesla-inc-0001318605-subject-3c6614d80c134a283be1671e8ed09176)
 Report of proposed sale of securities

[2024-04-25 20:34:00 Tesla Stock Rose Again. Three Reasons for the Jump.](https://newsfilter.io/articles/tesla-stock-rose-again-three-reasons-for-the-jump-d9ab871b77dd64fbf3ab9fab146e7b99)
 Tesla stock has risen for three consecutive days, after falling for seven straight. 

[2024-04-25 17:02:32 Deutsche Bank Maintains Hold on Tesla, Raises Price Target to \\$136](https://newsfilter.io/articles/deutsche-bank-maintains-hold-on-tesla-raises-price-target-to-136-4a8d0485a8505708997fbfbd9378e8b4)
 Deutsche Bank analyst Emmanuel Rosner maintains Tesla with a Hold and raises the price target from \\$123 to \\$136.

[2024-04-25 14:45:00 Stocks May Not Be Due for a Correction](https://newsfilter.io/articles/stocks-may-not-be-due-for-a-correction-759ab99c3df860dcae171b99e63642d2)
 The market may be bumpy for the next few months, but it probably won’t go down more than 10%.

[2024-04-25 14:09:00 Hertz’s Losses on Tesla EVs Bite Into Earnings](https://newsfilter.io/articles/hertzs-losses-on-tesla-evs-bite-into-earnings-2bfc6418cfd081b93f779ce6929460fb)
 The rental car company’s big push into EVs has proved to be a big problem.

[2024-04-25 13:28:00 Meta Shares Head for Worst Day in Years](https://newsfilter.io/articles/meta-shares-head-for-worst-day-in-years-88b530f3e7069ad6170a74d1f7c7c0e8)
 The company's plans to spend even more on its artificial-intelligence efforts appears to have shaken investors.

In [16]:
# open pages via selenium and firefox
outputdir = "htmldata"

# Print the formatted time
print(datetime.now().strftime('%H:%M:%S'), "Starting", flush=True)

firefox_app_path = '/Applications/Firefox.app'
# Path to your geckodriver
geckodriver_path = '/Users/drucev/webdrivers/geckodriver'

# Set up Firefox options to use your existing profile
# important for some sites that need a login, also a generic profile fingerprint that looks like a bot might get blocked
firefox_profile_path = '/Users/drucev/Library/Application Support/Firefox/Profiles/k8k0lcjj.default-release'
options = Options()
options.profile = firefox_profile_path

print(datetime.now().strftime('%H:%M:%S'), "Initialized profile", flush=True)

# Create a Service object with the path
service = Service(geckodriver_path)

print(datetime.now().strftime('%H:%M:%S'), "Initialized service", flush=True)
# Set up the Firefox driver
driver = webdriver.Firefox(service=service, options=options)

print(datetime.now().strftime('%H:%M:%S'), "Initialized webdriver", flush=True)
sleeptime = 10


10:46:26 Starting
10:46:38 Initialized profile
10:46:38 Initialized service
10:47:13 Initialized webdriver


In [17]:
# Open a new tab
def open_tab(driver, url):
    driver.execute_script(f"window.open('{url}');")
    # Switch to last one opened - sometimes hangs?
    # driver.switch_to.window(driver.window_handles[-1])
    # print(url)
    # time.sleep(sleeptime)  


In [18]:
url = f"https://stockcharts.com/h-sc/ui?s={symbol}&id=p33407302522&def=Y&listNum=1#"
source = "Stockcharts"
# Open the page
driver.get(url)

# Wait for the page to load
# time.sleep(sleeptime)  
display(Markdown(f"[{source}]({url})"))

[Stockcharts](https://stockcharts.com/h-sc/ui?s=TSLA&id=p33407302522&def=Y&listNum=1#)

In [19]:
url = f'https://www.bloomberg.com/search?query={company}'
source = "Bloomberg"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Bloomberg](https://www.bloomberg.com/search?query=Tesla)

In [20]:
url = f'https://www.reuters.com/site-search/?query={company}&offset=0'
source = "Reuters"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Reuters](https://www.reuters.com/site-search/?query=Tesla&offset=0)

In [21]:
url = f'https://finance.yahoo.com/quote/{symbol}/news'
source = "Yahoo quote"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Yahoo quote](https://finance.yahoo.com/quote/TSLA/news)

In [22]:
url = f'https://finance.yahoo.com/quote/{symbol}/key-statistics?ltr=1'
source = "Yahoo stats"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Yahoo stats](https://finance.yahoo.com/quote/TSLA/key-statistics?ltr=1)

In [23]:
url = f'https://www.ft.com/search?q={company}'
source = "FT"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[FT](https://www.ft.com/search?q=Tesla)

In [24]:
url = f'https://www.marketwatch.com/investing/stock/{symbol}?mod=search_symbol'
source = "Marketwatch"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Marketwatch](https://www.marketwatch.com/investing/stock/TSLA?mod=search_symbol)

In [25]:
url = f'https://www.barrons.com/market-data/stocks/{symbol}?mod=searchresults_companyquotes&mod=searchbar&search_keywords={company}&search_statement_type=suggested'    
source = "Barrons"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Barrons](https://www.barrons.com/market-data/stocks/TSLA?mod=searchresults_companyquotes&mod=searchbar&search_keywords=Tesla&search_statement_type=suggested)

In [26]:
url = f'https://www.businessinsider.com/s?q={company}'
source = "Insider"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Insider](https://www.businessinsider.com/s?q=Tesla)

In [27]:
url = f'https://www.google.com/finance/quote/{symbol}:{exchange}'
source = "Google Finance"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Google Finance](https://www.google.com/finance/quote/TSLA:NASDAQ)

In [28]:
url = f'https://finviz.com/quote.ashx?t={symbol}&p=d'
source = "FinViz"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[FinViz](https://finviz.com/quote.ashx?t=TSLA&p=d)

In [29]:
url = f'https://www.reddit.com/search?q={company}&include_over_18=on&sort=relevance&t=all'
source = "Reddit"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))

[Reddit](https://www.reddit.com/search?q=Tesla&include_over_18=on&sort=relevance&t=all)

In [30]:
morn_exch = morningstarmap[exchange]
url = f'https://www.morningstar.com/stocks/{morn_exch}/{symbol}/quote'
source = "Morningstar"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))
    

[Morningstar](https://www.morningstar.com/stocks/xnas/TSLA/quote)

In [31]:
url = f'https://whalewisdom.com/stock/{symbol}'
source = "WhaleWisdom"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))


[WhaleWisdom](https://whalewisdom.com/stock/TSLA)

In [32]:
url = f'https://www.gurufocus.com/stock/{symbol}/guru-trades'
source = "GuruFocus"
open_tab(driver, url)
display(Markdown(f"[{source}]({url})"))


[GuruFocus](https://www.gurufocus.com/stock/TSLA/guru-trades)

In [33]:
# https://www.barchart.com/stocks/highs-lows/lows?orderBy=highPercent1y&orderDir=asc
#     https://stockcharts.com/h-sc/ui?s=%24DJUSEN&id=p33407302522&def=Y&listNum=1
#         https://stockcharts.com/h-sc/ui?s=%24DJUSDN&id=p33407302522&def=Y&listNum=1

SyntaxError: invalid syntax (1428333994.py, line 1)